## **Laboratorio 4 - Data Science**
### *Mejorando el Análisis de Sentimientos con LSTM y Características Adicionales*
Stefano Aragoni, Carol Arévalo

----------

#### `Importación de Datos`
- Utilice el conjunto de datos IMDB proporcionado por Keras. pero esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes,utilice las 50.000 palabras más frecuentes


Como primer paso, se importan las diferentes librerías a utilizar. Principalmente se queriere de Keras, ya que el conjunto de datos IMBD se encuentra en esta librería. Además, se importan las librerías de numpy y pandas para el manejo de los datos.

In [9]:
import tensorflow as tf
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
import nltk

Posteriormente, se <font color='orange'>**importa el conjunto de datos con 50,000 palabras más frecuentes.**</font>

In [2]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

X_combined = list(X_train) + list(X_test)
y_combined = list(y_train) + list(y_test)
df = pd.DataFrame({'Text': X_combined, 'Label': y_combined})

print("Datos cargados.")

Cargando los datos...
17464789/17464789 [==============================] - 16s 1us/step
Datos cargados.


En este caso se optó por mezclar los datos de entrenamiento y prueba, para luego separarlos en 80% y 20% respectivamente. Esto debido a que originalmente estaban divididos como 50% de entrenamiento y 50% de prueba.

In [3]:
df.head()

,Text,Label
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",1
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",0
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",0
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",0


Más específicamente, como se puede observar a continuación, el dataset actual tiene los 50,000 datos. Por tal razón, **se logró la correcta importación del conjunto de datos con 50,000 palabras más frecuentes**.

In [4]:
print("Tamaño de los datos: ", df.shape)

Tamaño de los datos:  (50000, 2)


Con eso listo, se quiso convertir los datos a un formato más legible, por lo que se utilizó **word index** para convertir los mensajes tokenizados a texto. De tal manera, se puede analizar más a profundidad el contenido de los diferentes mensajes. 

In [5]:
word_index = imdb.get_word_index()
reverse_word_index = {index: word for word, index in word_index.items()}

def decode_sequence(sequence):
    decoded_words = [reverse_word_index.get(index - 3, '') for index in sequence]
    decoded_words = [word for word in decoded_words if word != '']
    return ' '.join(decoded_words)


decoded = [decode_sequence(seq) for seq in df['Text']]
decoded_df = pd.DataFrame({'Text': decoded, 'Label': df['Label']})

decoded_df.head()

1641221/1641221 [==============================] - 3s 2us/step


,Text,Label
0,this film was just brilliant casting location ...,1
1,big hair big boobs bad music and a giant safet...,0
2,this has to be one of the worst films of the 1...,0
3,the scots excel at storytelling the traditiona...,1
4,worst mistake of my life br br i picked this m...,0


#### `Pre-procesamiento`
- Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
- De las críticas, extraiga características (features) adicionales, por ejemplo. la longitud de la crítica, la proporción de palabras positivas/negativas y cualquier otra que considere pueda ser útil.

Como primer paso, se limpió el dataset para eliminar stopwords (palabras vacías o irrelevantes). Asimismo, se convirtió las palabras a su formato base (stemming) para evitar tener diferentes palabras con el mismo significado. Por ejemplo, "running" y "run" son palabras que tienen el mismo significado, por lo que se convirtieron a "run".

In [6]:
# Eliminar palabras vacías (stop words)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
decoded_df["Text"] = decoded_df["Text"].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Realizar lematización o stemming (opcional)
stemmer = PorterStemmer()
decoded_df["Text"] = decoded_df["Text"].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))

# Mostrar el DataFrame
decoded_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Text,Label
0,film brilliant cast locat sceneri stori direct...,1
1,big hair big boob bad music giant safeti pin w...,0
2,one worst film 1990 friend watch film target a...,0
3,scot excel storytel tradit sort mani year even...,1
4,worst mistak life br br pick movi target 5 fig...,0


Luego, se separaron los datos en el conjunto de entrenamiento y prueba. En este caso, se utilizó el 80% de los datos para entrenamiento y el 20% para prueba.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(decoded_df['Text'], decoded_df['Label'], test_size=0.2, random_state=42)

Posteriormente, se procedió a tokenizar los datos, es decir, convertir las palabras a números. Esto se hizo con el fin de poder utilizar los datos en el modelo de LSTM.

En otras palabras, se revirtió el proceso de word index, para convertir los mensajes a números. Esto fue necesario ya que se cambió el contenido de los mensajes a su formato base.

In [11]:
# Tokenización de texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_valid_sequences = tokenizer.texts_to_sequences(X_test)

Posteriormente, se <font color='orange'>**secuenció y rellenó los diferentes comentarios para que todos tengan una longitud uniforme**</font>. En este caso se utilizó una longitud 200. 

sequence.pad_sequences es una función de Keras que se utiliza para rellenar las secuencias. Si se pasa del largo máximo, se trunca la secuencia. Si es menor, se rellena con ceros.

In [ ]:
# Padding de secuencias para que tengan la misma longitud
max_length = 200  # longitud máxima de una secuencia
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_valid_padded = pad_sequences(X_valid_sequences, maxlen=max_length, padding='post', truncating='post')

En este caso, únicamente se <font color='orange'>**extrajo la proporción de palabras positivas y negativas**</font> de cada mensaje. Esto se hizo con el fin de poder utilizar dicha información como una característica adicional en el modelo de LSTM. A continuación **se justifica la razón de por qué se optó por esta característica**.

Se consideró que la proporición de palabras positivas y negativas es una característica que puede ser útil para el análisis de sentimientos. Esto considerando que si un mensaje tiene más palabras positivas que negativas, es probable que el sentimiento del mensaje sea positivo. Asimismo, si un mensaje tiene más palabras negativas que positivas, es probable que el sentimiento del mensaje sea negativo. 

De igual manera, cabe destacar que el modelo (aparte de la etiqueta) no sabe ni conoce el sentimiento de los mensajes. Por tal razón, al indicarle la proporción de palabras positivas y negativas, este modelo puede comprender más a fondo cómo la proporción de palabras positivas y negativas afecta el sentimiento de los mensajes.


Cabe destacar que no se utilizó otras características, como el largo de la crítica, ya que se consideró que esto no aportaba mucha información. Una reseña mala puede ser igual de larga o corta que una reseña buena. Por tal razón, se optó por utilizar únicamente la proporción de palabras positivas y negativas.

Como se puede observar a continuación, para esto se utilizó la librería textblob. Lo que se hizo fue pasarle a esta librería las palabras individuales de cada mensaje, y esta librería se encargó de identificar si la palabra es positiva o negativa. Posteriormente, se sumaron las palabras positivas y negativas, y se dividió entre el total de palabras del mensaje. De esta manera, se obtuvo la proporción de palabras positivas y negativas de cada mensaje.

Finalmente, se guardó dicha información en un dataframe, para luego utilizarlo como una característica adicional en el modelo de LSTM.

In [14]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

from textblob import TextBlob

def get_sentiment_ratio(text):
    words = text.split()
    positive_count = 0
    negative_count = 0
    
    for word in words:
        blob = TextBlob(word)
        if blob.sentiment.polarity > 0.5:  # Consideramos palabras con polaridad > 0.5 como positivas
            positive_count += 1
        elif blob.sentiment.polarity < -0.5:  # Consideramos palabras con polaridad < -0.5 como negativas
            negative_count += 1

    # Evitar la división por cero
    if negative_count == 0:
        return positive_count
    return positive_count / negative_count

# Calcular la proporción para cada comentario
decoded_df['Pos_Neg_Ratio'] = decoded_df['Text'].apply(get_sentiment_ratio)

# Mostrar el DataFrame con la nueva columna
print(decoded_df.head())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\carev\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\carev\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


                                                Text  Label  Pos_Neg_Ratio
0  film brilliant cast locat sceneri stori direct...      1       5.000000
1  big hair big boob bad music giant safeti pin w...      0       0.750000
2  one worst film 1990 friend watch film target a...      0       0.250000
3  scot excel storytel tradit sort mani year even...      1       1.000000
4  worst mistak life br br pick movi target 5 fig...      0       0.166667


#### `Modelo`
- Cree un modelo LSTM que acepte las características (features) adicionales junto con la secuencia de palabras.
- Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y tal vez alguna capa densamente conectada después de la LSTM. (ver también la referencia al final de este documento)

Como primer paso, se optó por normalizar la característica adicional seleccionada. Esta siendo la proporción de palabras positivas y negativas. Esto se hizo con el fin de que la característica adicional tenga un valor entre 0 y 1, lo cual es más fácil de manejar para el modelo.

Posteriormente, se dividió la característica adicional en el conjunto de entrenamiento y prueba.

In [ ]:
# 1. Preparar la característica adicional
# Normalizar la columna 'Pos_Neg_Ratio'
scaler = MinMaxScaler()
decoded_df['Normalized_Ratio'] = scaler.fit_transform(np.array(decoded_df['Pos_Neg_Ratio']).reshape(-1, 1))

# Dividir la característica adicional para conjuntos de entrenamiento y validación
X_train_ratio = np.array(decoded_df.loc[X_train.index, 'Normalized_Ratio'])
X_valid_ratio = np.array(decoded_df.loc[X_test.index, 'Normalized_Ratio'])

Con eso listo, se <font color=orange>**creó el modelo LSTM que acepta las características adicionales, así como las sencuencias de palabras.**</font>

Más específicamente, se utilizó **Functional** en vez de Sequential. Esto con el objetivo de poder utilizar múltiples entradas en el modelo. En este caso, se utilizaron dos entradas: la secuencia de palabras y la característica adicional.


Asimismo, a diferencia del ejemplo proporcionado por el profesor Luis Furlán, en este caso se <font color=orange>**usó una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y una capa densamente conectada después de la LSTM.**</font> A continuación se explica a más detalle la arquitectura utilizada.


- Como primer paso, se utilizó una capa de embedding con una dimensión de 128 para representar las palabras en la secuencia de entrada.
    - Se escogió esto ya que esta capa de embedding permite convertir las palabras en vectores de números reales que el modelo puede procesar.

- Luego, se añadieron dos capas LSTM; ambas con 64 neuronas. Una de estas capas tiene retorno de secuencias, mientras que la otra no.
    - Se escogió esto ya que las capas LSTM permiten que el modelo aprenda patrones a lo largo de la secuencia de palabras. Asimismo, se escogió una capa LSTM con retorno de secuencias y otra sin retorno de secuencias, ya que la primera capa permite que el modelo procese y capture información a lo largo de toda la secuencia de palabras, mientras que la segunda capa condensa esta información en una representación final.

- Para evitar el sobreajuste del modelo, se incluyeron capas de Dropout con una tasa del 50% después de cada capa LSTM.
    - Se seleccionó Dropout como técnica de regularización ya que esta capa permite que el modelo ignore aleatoriamente el 50% de las neuronas de la capa anterior. Esto con el fin de evitar que el modelo se sobreajuste a los datos de entrenamiento.


- Posteriormente, se incluyó otra entrada para la característica adicional. Se combinaron ambas entradas con una capa de concatenación.
    - La característica adicional se incluyó como una entrada adicional ya que esta información puede ser útil para el modelo. Por ejemplo, si un mensaje tiene más palabras positivas que negativas, es probable que el sentimiento del mensaje sea positivo. Asimismo, si un mensaje tiene más palabras negativas que positivas, es probable que el sentimiento del mensaje sea negativo.

- Finalmente, se añadieron capas densamente conectadas para procesar esta representación combinada. La primera capa densamente conectada utiliza una función de activación ReLU. Luego, se aplicó una capa de Dropout nuevamente para la regularización del modelo. La capa de salida utiliza una función de activación sigmoide para predecir una etiqueta binaria.
    - Se seleccionó ReLU como función de activación ya que esta función permite que el modelo aprenda patrones más complejos en los datos. Asimismo, se seleccionó sigmoide como función de activación en la capa de salida ya que esta función permite que el modelo prediga una etiqueta binaria, que en este contexto podría ser una clasificación de sentimiento positivo o negativo.

In [ ]:
# 2. Construir el modelo LSTM con características adicionales
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128

# Entrada de secuencia de palabras
input_seq = Input(shape=(max_length,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_seq)
lstm_layer1 = LSTM(64, return_sequences=True)(embedding_layer)
dropout_layer1 = Dropout(0.5)(lstm_layer1)
lstm_layer2 = LSTM(64)(dropout_layer1)

# Entrada de características adicionales
input_features = Input(shape=(1,))
concat_layer = Concatenate()([lstm_layer2, input_features])

# Capas densamente conectadas
dense_layer1 = Dense(64, activation='relu')(concat_layer)
dropout_layer2 = Dropout(0.5)(dense_layer1)
output_layer = Dense(1, activation='sigmoid')(dropout_layer2)

#### `Entrenamiento y Evaluación`
- Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.

Posteriormente, <font color=orange>**se entrenó al respectivo modelo utilizando los valores anteriormente preparados**</font>; incluyendo las secuencias de palabras así como la característica adicional (proporción de palabras positivas/total). Como se puede observar, **no hubo errores en el proceso de entrenamiento**. 

In [15]:
model = Model(inputs=[input_seq, input_features], outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit([X_train_padded, X_train_ratio], y_train, validation_data=([X_valid_padded, X_valid_ratio], y_test), epochs=5, batch_size=64)


Epoch 1/5
625/625 [==============================] - 203s 318ms/step - loss: 0.6683 - accuracy: 0.5958 - val_loss: 0.6490 - val_accuracy: 0.6007
Epoch 2/5
625/625 [==============================] - 224s 359ms/step - loss: 0.6323 - accuracy: 0.6191 - val_loss: 0.4943 - val_accuracy: 0.7826
Epoch 3/5
625/625 [==============================] - 220s 352ms/step - loss: 0.5589 - accuracy: 0.7174 - val_loss: 0.4103 - val_accuracy: 0.8295
Epoch 4/5
625/625 [==============================] - 225s 360ms/step - loss: 0.3064 - accuracy: 0.8791 - val_loss: 0.2827 - val_accuracy: 0.8842
Epoch 5/5
625/625 [==============================] - 211s 338ms/step - loss: 0.1861 - accuracy: 0.9347 - val_loss: 0.2799 - val_accuracy: 0.8918


Finalmente, <font color=orange>**se evaluó el desempeño final del modelo utilizando el conjunto de prueba (20% de los datos).**</font> Como se puede observar a continuación, se obtuvo un 89.18% de accuracy. Esto quiere decir que el modelo logró predecir correctamente el sentimiento de los mensajes en un 89.18% de los casos.
 
Cabe destacar que el modelo realizado por el Profesor obtuvo, apenas, un 80% de precisión. Por tal razón, se puede concluir que el modelo diseñado logró incrementar la precisión en el análisis de sentimientos sobre las críticas de películas. *Sin embargo, el modelo original utilizaba una arquitectura Sequential en vez de Functional*. En realidad, el Profesor indicó que intentó utilizar una arquitectura Functional, pero que con este obtuvo una precisión de 50%.

En base a lo anteriormente mencionado, se puede concluir que de 50% a 89% es un incremento significativo en la precisión del modelo. Por tal razón, se puede concluir que el modelo fue diseñado de manera correcta.


Cabe destacar que la **ventaja del modelo realizado es que este utilizó una característica adicional, la cual fue la proporción de palabras positivas y negativas**. Esto fue beneficioso, ya que el modelo pudo comprender más a fondo cómo la proporción de palabras positivas y negativas afecta el sentimiento de los mensajes. Asimismo, posiblemente existía un patrón entre la proporción de palabras positivas y negativas y el sentimiento de los mensajes. Por tal razón, se puede concluir que la característica adicional fue de gran ayuda para el modelo.

In [1]:
perdida, exactitud = model.evaluate([X_valid_padded, X_valid_ratio], y_test, batch_size = 64, verbose = 0)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

Pérdida de la Prueba: 0.2799
Exactitud de la Prueba (Test accuracy): 0.8918


*Según Furlán, el Jupyter Notebook cuenta como el informe en sí. Se incluyó las 3 partes indicadas.*